In [1]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!pip install line-bot-sdk

DEPRECATION: Loading egg at c:\users\earth\appdata\local\programs\python\python311\lib\site-packages\jinja2-3.1.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\earth\appdata\local\programs\python\python311\lib\site-packages\jinja2-3.1.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\earth\appdata\local\programs\python\python311\lib\site-packages\jinja2-3.1.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
user_ids = []

In [3]:
!ngrok authtoken 2Wcl8tU0b0zfsRlmhbJfMKgUcDe_3yLPnFxKn87DWX82hdAK7

from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

import json
import random
import time
import json
import requests
import datetime
from zoneinfo import ZoneInfo

safe_level_message = {
    "type":"text",
    "text":"$ อยู่ในระดับที่ปลอดภัย\n\nสามารถทำกิจกรรมกลางแจ้งและการท่องเที่ยวได้ตามปกติ",
    "emojis": [
        {
            "index": 0,
            "productId": "5ac21a18040ab15980c9b43e",
            "emojiId": "028",
        },
    ]
}
warning_level_message = {
    "type":"text",
    "text":"$ อยู่ในระดับที่ต้องเฝ้าระวัง\n\nควรเฝ้าระวังสุขภาพ ถ้ามีอาการเกี่ยวกับระบบทางเดินหายใจ ระคายเคืองตา ไม่ควรทำกิจกรรมกลางแจ้งนาน หรือใช้อุปกรณ์ป้องกัน ส่วนผู้ที่ต้องดูแลสุขภาพเป็นพิเศษ แล้วมีอาการทางสุขภาพ เช่น ไอ หายใจลำบาก ตาอักเสบ แน่นหน้าอก ปวดศีรษะ หัวใจเต้นไม่เป็นปกติ คลื่นไส้ อ่อนเพลีย ควรปรึกษาแพทย์",
    "emojis": [
        {
            "index": 0,
            "productId": "5ac21a18040ab15980c9b43e",
            "emojiId": "047",
        },
    ]
}
danger_level_message = {
    "type":"text",
    "text":"$ อยู่ในระดับอันตรายอย่างมาก\n\nควรหลีกเลี่ยงกิจกรรมกลางแจ้งทุกอย่างหลีกเลี่ยงพื้นที่ที่มีมลพิษทางอากาศสูง หรือใช้อุปกรณ์ป้องกันตนเองหากมีความจำเป็น หากมีอาการทางสุขภาพควรปรึกษาแพทย์",
    "emojis": [
        {
            "index": 0,
            "productId": "5ac21a18040ab15980c9b43e",
            "emojiId": "048",
        },
    ]
}

quickReplyMessage = {
    "type": "text",
    "text": "เลือกตรวจสอบค่าฝุ่นได้เลย!",
    "quickReply": {
        "items": [
        {
            "type": "action",
            "action": {
            "type": "message",
            "label": "ค่าฝุ่นล่าสุด",
            "text": "ค่าฝุ่นล่าสุด"
            }
        },
        {
            "type": "action",
            "action": {
            "type": "message",
            "label": "ย้อนหลัง 1 ชั่วโมง",
            "text": "ย้อนหลัง 1 ชั่วโมง"
            }
        },
        {
            "type": "action",
            "action": {
            "type": "message",
            "label": "24 ชั่วโมง",
            "text": "24 ชั่วโมง"
            }
        },
        ]
    }
}


def handler_reply_msg(headers, json_data, line_bot_api):
    try:
        userId = json_data['events'][0]['source']['userId']
        if userId not in user_ids:
            user_ids.append(userId)
            print("add user: ", userId)

        requestMessage = json_data['events'][0]['message']['text']
        push_url = "https://api.line.me/v2/bot/message/push"

        if requestMessage == "ค่าฝุ่นล่าสุด":
            thingspeak_url = "https://api.thingspeak.com/channels/2273773/feeds.json?results=1"
            thingspeak_response = requests.get(thingspeak_url)
            thingspeak_feed = json.loads(thingspeak_response.text)['feeds'][0]
            created_at = thingspeak_feed['created_at']
            created_date = datetime.datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%S%z').astimezone(ZoneInfo('Asia/Bangkok'))
            created_string = created_date.strftime("%H:%M:%S %d %b %Y ")

            pm1_0 = thingspeak_feed['field1']
            pm2_5 = thingspeak_feed['field2']
            pm10_0 = thingspeak_feed['field3']

            if float(pm2_5) < 50:
                level_message = safe_level_message
            elif float(pm2_5) < 100:
                level_message = warning_level_message
            else:
                level_message = danger_level_message

            message_json = {
                "to": userId,
                "messages":[
                    {
                        "type":"text",
                        "text": f"$ วัดล่าสุดเมื่อเวลา\n{created_string}",
                        "emojis": [
                            {
                                "index": 0,
                                "productId": "5ac21a18040ab15980c9b43e",
                                "emojiId": "192",
                            },
                        ]
                    },
                    {
                        "type":"text",
                        "text": f"$ ค่าล่าสุดที่วัดได้\n\n\tPM1.0:  \t{pm1_0} µg/m3\n\tPM2.5:  \t{pm2_5} µg/m3\n\tPM10.0:\t{pm10_0} µg/m3",
                        "emojis": [
                            {
                                "index": 0,
                                "productId": "5ac21a18040ab15980c9b43e",
                                "emojiId": "129",
                            },
                        ]
                    },
                    level_message,
                    quickReplyMessage,
                ]
            }
            response = requests.post(push_url, headers=headers, json=message_json)
            print(response.text)

            return True

        elif requestMessage == "ย้อนหลัง 1 ชั่วโมง":
            thingspeak_url = "https://api.thingspeak.com/channels/2273773/feeds.json?results=240"
            thingspeak_response = requests.get(thingspeak_url)
            thingspeak_feeds = json.loads(thingspeak_response.text)['feeds']

            total_pm1_0 = []
            total_pm2_5 = []
            total_pm10_0 = []
            for feed in thingspeak_feeds:
                total_pm1_0.append(float(feed['field1']))
                total_pm2_5.append(float(feed['field2']))
                total_pm10_0.append(float(feed['field3']))

            average_pm1_0 = sum(total_pm1_0) /len(total_pm1_0)
            average_pm2_5 = sum(total_pm2_5) /len(total_pm2_5)
            average_pm10_0 = sum(total_pm10_0) /len(total_pm10_0)

            if average_pm2_5 < 50:
                level_message = safe_level_message
            elif average_pm2_5 < 100:
                level_message = warning_level_message
            else:
                level_message = danger_level_message

            message_json = {
                "to": userId,
                "messages":[
                    {
                        "type":"text",
                        "text": "$ ค่าฝุ่นเฉลี่ยย้อนหลัง 1 ชั่วโมง",
                        "emojis": [
                            {
                                "index": 0,
                                "productId": "5ac21a18040ab15980c9b43e",
                                "emojiId": "192",
                            },
                        ]
                    },
                    {
                        "type":"text",
                        "text": f"$ ค่าเฉลี่ยทั้งหมด\n\n\tPM1.0:  \t{average_pm1_0:.2f} µg/m3\n\tPM2.5:  \t{average_pm2_5:.2f} µg/m3\n\tPM10.0:\t{average_pm10_0:.2f} µg/m3",
                        "emojis": [
                            {
                                "index": 0,
                                "productId": "5ac21a18040ab15980c9b43e",
                                "emojiId": "129",
                            },
                        ]
                    },
                    level_message,
                    quickReplyMessage,
                ]
            }
            response = requests.post(push_url, headers=headers, json=message_json)
            print(response.text)
            return True

        elif requestMessage == "24 ชั่วโมง":
            thingspeak_url = "https://api.thingspeak.com/channels/2273773/feeds.json?results=5760"
            thingspeak_response = requests.get(thingspeak_url)
            thingspeak_feeds = json.loads(thingspeak_response.text)['feeds']

            total_pm1_0 = []
            total_pm2_5 = []
            total_pm10_0 = []
            for feed in thingspeak_feeds:
                total_pm1_0.append(float(feed['field1']))
                total_pm2_5.append(float(feed['field2']))
                total_pm10_0.append(float(feed['field3']))

            average_pm1_0 = sum(total_pm1_0) /len(total_pm1_0)
            average_pm2_5 = sum(total_pm2_5) /len(total_pm2_5)
            average_pm10_0 = sum(total_pm10_0) /len(total_pm10_0)

            if average_pm2_5 < 50:
                level_message = safe_level_message
            elif average_pm2_5 < 100:
                level_message = warning_level_message
            else:
                level_message = danger_level_message

            message_json = {
                "to": userId,
                "messages":[
                    {
                        "type":"text",
                        "text": "$ 24 ชั่วโมงที่ผ่านมา",
                        "emojis": [
                            {
                                "index": 0,
                                "productId": "5ac21a18040ab15980c9b43e",
                                "emojiId": "192",
                            },
                        ]
                    },
                    {
                        "type":"text",
                        "text": f"$ ค่าเฉลี่ยทั้งหมด\n\n\tPM1.0:  \t{average_pm1_0:.2f} µg/m3\n\tPM2.5:  \t{average_pm2_5:.2f} µg/m3\n\tPM10.0:\t{average_pm10_0:.2f} µg/m3",
                        "emojis": [
                            {
                                "index": 0,
                                "productId": "5ac21a18040ab15980c9b43e",
                                "emojiId": "129",
                            },
                        ]
                    },
                    level_message,
                    quickReplyMessage,
                ]
            }
            response = requests.post(push_url, headers=headers, json=message_json)
            print(response.text)
            return True

        else:
            message_json = {
                "to": userId,
                "messages": [quickReplyMessage]
            }
            response = requests.post(push_url, headers=headers, json=message_json)
            print(response.text)
            return True

    except Exception as error:
        print("An exception occurred:", error)

Authtoken saved to configuration file: C:\Users\earth/.ngrok2/ngrok.yml


In [4]:
app = Flask(__name__)

@app.route("/user_ids", methods=['GET'])
def get_user_ids():
    global user_ids
    return jsonify({'user_ids': user_ids})

@app.route("/", methods=['POST'])
def linebot():
    global user_ids
    body = request.get_data(as_text=True)

    json_data = json.loads(body)
    access_token = 'vM/06TZq7lbmdS1xG0eopxNBkDcNNf0035dP95pNbgpvt31fvgyB7g86xkpDaRH6bYysY/v/OxgLYQwkqDF2/hdQuHHUHvdtPPOlsMvWqtIrO54CkLr2j2xx3MDE5z001+p6XJNQRNJOPgL704QAVgdB04t89/1O/w1cDnyilFU='
    secret = '4cc223fce8a12ca9f4566c0280952ac0'
    line_bot_api = LineBotApi(access_token)
    handler = WebhookHandler(secret)
    signature = request.headers['X-Line-Signature']
    handler.handle(body, signature)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + access_token
    }
    handler_reply_msg(headers, json_data, line_bot_api)
    return 'OK'

if __name__ == "__main__":
  run_with_ngrok(app)
  app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-13:
Traceback (most recent call last):
  File "c:\Users\earth\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\earth\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1394, in run
    self.function(*self.args, **self.kwargs)
  File "c:\Users\earth\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
                    ^^^^^^^^^^^^
  File "c:\Users\earth\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask_ngrok.py", line 38, in _run_ngrok
    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
                 ~~~~~~~~~~~~^^^
IndexError: list index out of range
